In [1]:
%run GpuOptions.ipynb
%run BuildTrainingData.ipynb
%run TrendTools.ipynb

Found 2929 images belonging to 4 classes.
Found 731 images belonging to 4 classes.


In [4]:
import os
import time
import shutil
import tensorflow as tf
import tensorflow_hub as hub
import model as m

classes = training_data_generator.class_indices
data, _ = training_data_generator.next()
shape = data[0].shape
inputs = tf.keras.Input(shape=shape)
outputs = m.chained(
    tf.keras.layers.add([
        m.chained(
            inputs,
            tf.keras.layers.Lambda(
                lambda image: tf.image.rgb_to_grayscale(image)),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, activation='relu'),
            tf.keras.layers.MaxPooling2D(2),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, activation='relu'),
            tf.keras.layers.MaxPooling2D(2),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, activation='relu'),
            tf.keras.layers.MaxPooling2D(2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(32),
        ),
        m.chained(
            inputs,
            tf.keras.layers.Lambda(
                lambda image: tf.image.rgb_to_grayscale(image)),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, activation='relu'),
            tf.keras.layers.MaxPooling2D(2),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, activation='relu'),
            tf.keras.layers.MaxPooling2D(2),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, activation='relu'),
            tf.keras.layers.MaxPooling2D(2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(32),
        ),
    ]),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(classes)),
    tf.keras.layers.Activation('softmax'))
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.build(input_shape=(None, *shape))
model.compile(
    optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.01),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, label_smoothing=0.05),
    metrics=['accuracy'])
if os.path.exists('isthemountainout.best.h5'):
    model.load_weights('isthemountainout.best.h5')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 224, 224, 1)  0           input_2[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 224, 224, 1)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 222, 222, 64) 640         lambda_2[0][0]                   
____________________________________________________________________________________________

In [5]:
!rm -rf logs
time.sleep(2)

model.fit(
    training_data_generator,
    epochs=700,
    verbose=True,
    steps_per_epoch=training_data_generator.samples // 
        training_data_generator.batch_size,
    validation_data=validation_data_generator,
    validation_steps=validation_data_generator.samples // 
        validation_data_generator.batch_size,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            'isthemountainout.best.h5',
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=True),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            mode='min',
            patience=50,
            restore_best_weights=True,
            verbose=True),
        tf.keras.callbacks.CSVLogger(os.path.join('logs', 'isthemountainout.training.csv')),
        tf.keras.callbacks.TensorBoard(
            log_dir='logs', update_freq='batch', write_images=True, write_graph=True, embeddings_freq=10),
        m.LogConfusionMatrixCallback(
            model=model,
            datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)\
                .flow_from_directory(data_directory, batch_size=3096, shuffle=True, target_size=image_size),
            logdir=os.path.join('logs', 'image'))
    ])

=========================] - 48s 527ms/step - loss: 0.8526 - accuracy: 0.9320 - val_loss: 0.9016 - val_accuracy: 0.8793
Epoch 631/700
90/91 [============================>.] - ETA: 0s - loss: 0.8551 - accuracy: 0.9312
Epoch 00631: val_accuracy did not improve from 0.88636
91/91 [==============================] - 69s 759ms/step - loss: 0.8548 - accuracy: 0.9313 - val_loss: 0.9087 - val_accuracy: 0.8622
Epoch 632/700
90/91 [============================>.] - ETA: 0s - loss: 0.8543 - accuracy: 0.9271
Epoch 00632: val_accuracy did not improve from 0.88636
91/91 [==============================] - 47s 515ms/step - loss: 0.8544 - accuracy: 0.9268 - val_loss: 0.8971 - val_accuracy: 0.8736
Epoch 633/700
90/91 [============================>.] - ETA: 0s - loss: 0.8569 - accuracy: 0.9236
Epoch 00633: val_accuracy did not improve from 0.88636
91/91 [==============================] - 69s 756ms/step - loss: 0.8569 - accuracy: 0.9237 - val_loss: 0.8996 - val_accuracy: 0.8778
Epoch 634/700
90/91 [=======